In [13]:
import sys
from functools import partial

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim.lr_scheduler as lr_scheduler
import wandb
from ignite.contrib.handlers import wandb_logger
from ignite.engine import (Engine, Events, create_supervised_evaluator,
                           create_supervised_trainer)
from ignite.handlers import ModelCheckpoint
from ignite.handlers.param_scheduler import LRScheduler
from ignite.metrics import Accuracy, Loss
from scipy.io.arff import loadarff
from sklearn.model_selection import train_test_split
from torch import nn
from torch.functional import F
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler

sys.path.append('../')
from src.datasets import FordDataset
from src.models import TransformerClassification
from src.utils import build_optimizer, str2torch

In [14]:
config = {
    "model":{
        "embedding":{
            "in_channels": 1,
            "out_channels": 16,
        },
        "encoder": {
            "layer_norm_eps": 5e-4,
            "dropout": 0.5,
            "activation": 'elu'
        },
        "num_layers": 1,
        "fc":{
            "inner_dim": 50,
            "dropout": 0.5
        }
    },
    "train":{
        "optimizer": 'adam',
        "lr": 3e-4,
        "n_epoch": 2,
        "scheduler_config": {
            "type": None,
        }
    },
    "data":{
        "seq_length": 500,
        "step": 25
    },
    "random_state": 42
}

In [15]:
train_path = "../data/FordA/FordA_TRAIN.arff"
test_path = "../data/FordA/FordA_TEST.arff"

train_dataset = FordDataset(train_path, config['data'])
test_dataset = FordDataset(test_path, config['data'])

idx = np.arange(len(train_dataset))
idx_train, idx_val = train_test_split(idx, train_size=0.8, stratify=train_dataset.labels, random_state=config['random_state'])

train_sampler = SubsetRandomSampler(idx_train)
val_sampler = SubsetRandomSampler(idx_val)

train_dataloader = DataLoader(train_dataset, batch_size=128, sampler=train_sampler)
val_dataloader = DataLoader(train_dataset, batch_size=128, sampler=val_sampler)
test_dataloader = DataLoader(test_dataset, batch_size=64)

In [16]:
# Initialize your model
wandb.init(entity='ts-robustness', project='ml-course', config=config, tags=['hypersearch'])

device = 'cuda' if torch.cuda.is_available() else 'cpu'
config['model']['encoder']['activation'] = str2torch(config['model']['encoder']['activation'])
config['train']['optimizer'] = str2torch(config['train']['optimizer'])

model = TransformerClassification(config).to(device)

# Initialize your optimizer and criterion
optimizer = build_optimizer(config, model)
criterion = nn.BCELoss()

def train_step(engine, batch):
    model.train()
    optimizer.zero_grad()
    x, y = batch[0].to(device), batch[1].to(device)
    y_pred = model(x)
    loss = criterion(y_pred, y.unsqueeze(1))
    loss.backward()
    optimizer.step()
    return loss.item()

trainer = Engine(train_step)

def validation_step(engine, batch):
    model.eval()
    with torch.no_grad():
        x, y = batch[0].to(device), batch[1].to(device)
        y_pred = model(x)
        return y_pred, y

train_evaluator = Engine(validation_step)
val_evaluator = Engine(validation_step)
test_evaluator = Engine(validation_step)

# Attach metrics to the evaluators
metrics = {
    'accuracy': Accuracy(output_transform=lambda x: (x[0] > 0.5, x[1])),
    'loss': Loss(criterion, output_transform=lambda x: (x[0], x[1].unsqueeze(1)))
}

for name, metric in metrics.items():
    metric.attach(train_evaluator, name)
    metric.attach(val_evaluator, name)
    metric.attach(test_evaluator, name)


# checkpoint_handler = ModelCheckpoint(dirname='saved_models', filename_prefix='best',
#                                      n_saved=1, require_empty=False,
#                                      score_function=lambda engine: engine.state.metrics['accuracy'],
#                                      score_name="accuracy", global_step_transform=lambda *_: trainer.state.epoch)
# val_evaluator.add_event_handler(Events.EPOCH_COMPLETED, checkpoint_handler, {"model": model})


@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
    train_evaluator.run(train_dataloader)
    metrics = train_evaluator.state.metrics
    print("Training Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.4f}"
          .format(trainer.state.epoch, metrics['accuracy'], metrics['loss']))
    wandb.log({"train_accuracy": metrics['accuracy'],
               "train_loss": metrics['loss']})

@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(trainer):
    val_evaluator.run(val_dataloader)
    metrics = val_evaluator.state.metrics
    print("Validation Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.4f}"
          .format(trainer.state.epoch, metrics['accuracy'], metrics['loss']))
    wandb.log({"val_accuracy": metrics['accuracy'],
               "val_loss": metrics['loss']})
    
@trainer.on(Events.COMPLETED)
def log_test_results(trainer):
    test_evaluator.run(test_dataloader)
    metrics = test_evaluator.state.metrics
    print("Test Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.4f}"
          .format(trainer.state.epoch, metrics['accuracy'], metrics['loss']))
    wandb.log({"test_accuracy": metrics['accuracy'],
               "test_loss": metrics['loss']})


# Run the training loop
trainer.run(train_dataloader, max_epochs=config['train']['n_epoch'])
wandb.finish()

train_accuracy,▁▆▇██
train_loss,█▄▂▁▁
val_accuracy,▁▆▇██
val_loss,█▄▂▁▁
train_accuracy,0.87415
train_loss,0.30408
val_accuracy,0.85523
val_loss,0.33093


Training Results - Epoch: 1  Avg accuracy: 0.69 Avg loss: 0.6279
Validation Results - Epoch: 1  Avg accuracy: 0.64 Avg loss: 0.6451
Training Results - Epoch: 2  Avg accuracy: 0.73 Avg loss: 0.5561
Validation Results - Epoch: 2  Avg accuracy: 0.68 Avg loss: 0.6006
Test Results - Epoch: 2  Avg accuracy: 0.67 Avg loss: 0.6020


test_accuracy,▁
test_loss,▁
train_accuracy,▁█
train_loss,█▁
val_accuracy,▁█
val_loss,█▁
test_accuracy,0.6697
test_loss,0.60202
train_accuracy,0.7309
train_loss,0.55614
val_accuracy,0.67684


In [17]:
# Count the number of parameters in the transformer_encoder layer
transformer_encoder_params = sum(p.numel() for p in model.transformer_encoder.parameters())

# Count the number of parameters in the fc layer
fc_params = sum(p.numel() for p in model.fc.parameters())

print("Number of parameters in transformer_encoder:", transformer_encoder_params)
print("Number of parameters in fc layer:", fc_params)


Number of parameters in transformer_encoder: 68752
Number of parameters in fc layer: 400101
